In [4]:
import staintools
import csv
import os
import glob
import re
from pandas import DataFrame, Series
from PIL import Image
import timeit
import time
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [5]:
train_paths = ["/scratch/kk4ze/data_lowres/train/Celiac/"]

In [6]:
# get images
images = {}
images_by_folder = {}
for train_path in train_paths:
    images_by_folder[str(train_path)] = []
    files = glob.glob(os.path.join(train_path, '*.jpg'))
    for fl in files:
        flbase = os.path.basename(fl)
        flbase_noext = os.path.splitext(flbase)[0]
        images[flbase_noext]=fl
        images_by_folder[str(train_path)].append(flbase_noext)

In [7]:
# initialize stain and brightness normalizer
stain_normalizer = staintools.StainNormalizer(method='vahadane')
standardizer = staintools.BrightnessStandardizer()

In [8]:
# choose target image
target_image = staintools.read_image("/scratch/kk4ze/data_lowres/train/Celiac/C03-05_03_5901_4803_horiz__0.jpg")
standard_target_image = standardizer.transform(target_image)
stain_normalizer.fit(standard_target_image)

In [9]:
# get destination path
path_change_map = {}

for key in list(images_by_folder.keys()):
    temp = key.replace('data_lowres', 'data_lowres_augmented')
    path_change_map[key] = temp

In [10]:
for key in images_by_folder.keys():
    for value in list(images_by_folder[key]):
#         print(key)
#         print(value)
#         print (str(count) + ' ' + str(value))
        source_img_path = str(key) + str(value) + '.jpg'
        dest_img_path = str(path_change_map[key]) + str(value) + '.jpg'
#         print(source_img_path)
        img = staintools.read_image(source_img_path)
        if (np.mean(img) > 240) or (np.mean(img) < 10):
            continue
        # standardize brightness
        img_standard = standardizer.transform(img)
        # transform the images
        img_normalized = stain_normalizer.transform(img_standard)
        # write image to path

#             plt.imshow(img)
#             plt.title('my picture')
#             plt.show()
#             plt.imshow(img_normalized)
#             plt.title('my picture')
#             plt.show()

      #  if (count % 1000 == 0):
#             plt.imshow(img)
#             plt.title('my picture')
#             plt.show()
#             plt.imshow(img_normalized)
#             plt.title('my picture')
#             plt.show()
        cv2.imwrite(os.path.normpath(dest_img_path), img_normalized)

In [ ]:
path_change_map